In [ ]:
!pip -q install ultralytics

In [ ]:
import os
import json
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from tqdm import tqdm
import torch

In [ ]:
IMAGES_DIR = "/content/drive/MyDrive/fall.v1i.coco/train"
COCO_JSON_PATH = "/content/drive/MyDrive/fall.v1i.coco/train/_annotations.coco.json"
OUTPUT_LABELS_DIR = "/content/drive/MyDrive/fall.v1i.coco"

In [ ]:
with open(COCO_JSON_PATH, 'r') as f:
    coco_data = json.load(f)

# Build a mapping: image_id → list of ground-truth fall bounding boxes (in xywh format)
image_id_to_bboxes = {}
for ann in coco_data["annotations"]:
    img_id = ann["image_id"]
    # COCO bbox format: [x_min, y_min, width, height]
    bbox = ann["bbox"]
    if img_id not in image_id_to_bboxes:
        image_id_to_bboxes[img_id] = []
    image_id_to_bboxes[img_id].append(bbox)

# Also build image_id → file_name mapping
image_id_to_filename = {img["id"]: img["file_name"] for img in coco_data["images"]}


In [ ]:
print("Loading YOLOv8n-pose model...")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = YOLO("yolov8n-pose.pt")  # Model will auto-use GPU if available


In [ ]:
def box_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xa1, ya1, xa2, ya2 = x1, y1, x1 + w1, y1 + h1
    xb1, yb1, xb2, yb2 = x2, y2, x2 + w2, y2 + h2

    inter_x1 = max(xa1, xb1)
    inter_y1 = max(ya1, yb1)
    inter_x2 = min(xa2, xb2)
    inter_y2 = min(ya2, yb2)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    if inter_area == 0:
        return 0.0

    area1 = w1 * h1
    area2 = w2 * h2
    union_area = area1 + area2 - inter_area

    return inter_area / union_area

In [ ]:
print(f"Processing {len(coco_data['images'])} images...")

for img_info in tqdm(coco_data["images"], desc="Pseudo-labeling"):
    image_id = img_info["id"]
    file_name = img_info["file_name"]
    img_path = Path(IMAGES_DIR) / file_name

    if not img_path.exists():
        print(f"⚠️  Warning: {img_path} not found. Skipping.")
        continue

    # Run inference on GPU (if available)
    results = model(img_path, device=device, verbose=False)
    if not results[0].boxes:
        continue

    # Move tensors to CPU only when needed for numpy conversion
    pred_boxes = results[0].boxes.xywh.cpu().numpy()  # [xc, yc, w, h] normalized
    pred_keypoints = results[0].keypoints.xyn.cpu().numpy()  # normalized [x, y]
    orig_img_shape = results[0].orig_shape  # (height, width)

    gt_boxes = image_id_to_bboxes.get(image_id, [])
    label_lines = []

    for i in range(len(pred_boxes)):
        xc, yc, w, h = pred_boxes[i]
        pred_box_abs = [
            (xc - w / 2) * orig_img_shape[1],  # x_min
            (yc - h / 2) * orig_img_shape[0],  # y_min
            w * orig_img_shape[1],             # width
            h * orig_img_shape[0]              # height
        ]

        keep = False
        if gt_boxes:
            for gt_box in gt_boxes:
                iou = box_iou(pred_box_abs, gt_box)
                if iou > 0.5:
                    keep = True
                    break
        else:
            keep = True

        if not keep:
            continue

        # Keypoints are already normalized (x, y in [0,1])
        kpts = pred_keypoints[i]
        kpt_list = []
        for j in range(17):
            x, y = kpts[j]
            vis = 2 if (0 <= x <= 1 and 0 <= y <= 1) else 0
            kpt_list.extend([x, y, vis])

        line = "0 " + f"{xc:.6f} {yc:.6f} {w:.6f} {h:.6f} " + " ".join(f"{kp:.6f}" for kp in kpt_list)
        label_lines.append(line)

    # Save label file
    label_file = Path(OUTPUT_LABELS_DIR) / (Path(file_name).stem + ".txt")
    with open(label_file, "w") as f:
        f.write("\n".join(label_lines))

print("✅ Pseudo-labeling complete!")
print(f"Labels saved to: {OUTPUT_LABELS_DIR}")